# Trabajo Práctico 3

Mariana Szewach - Camila Cirignoli - Amalia Oxandaberro

# Librerías

En primer lugar, se cargan todas las librerias a utilizar en el siguiente trabajo

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from string import punctuation
from termcolor import colored
from collections import Counter
!pip install UpSampling2D
from tensorflow.keras.layers import Dense, Flatten, Conv2D, UpSampling2D, Dropout,BatchNormalization,GlobalAveragePooling2D, Activation, MaxPooling2D, AveragePooling2D, Input, Add

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model

import math, keras, datetime, pandas as pd, numpy as np, keras.backend as K
import matplotlib.pyplot as plt, operator, random, pickle
!pip install utils2
from utils2 import *
!pip install pandas_summary
from pandas_summary import DataFrameSummary
from IPython.display import SVG
from numpy import mean, sqrt, square, arange
from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50

# Datos

Se cargan los datos de la competencia

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
x_train = np.load("../input/cnn-itba-2021-q2/X_train.npy")
y_train = np.load("../input/cnn-itba-2021-q2/y_train.npy")
x_test = np.load("../input/cnn-itba-2021-q2/X_test.npy")

# Pre procesamiento
Inicialmente, se separan los datos en train y valid para entrenar y validar el algoritmo respectivamente. Se dividieron los datos obteniendo un 80% para train y un 20% para valid. 

In [22]:
valid_porcentage = 0.2
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=valid_porcentage, random_state=42, stratify=y_train)


A continuación se normalizan los datos para obtener valores entre 0 y 1. 
Además se generan los one-hot encoding para convertir las etiquetas de las clases a vectores de 0s y 1s.

In [23]:
x_train_norm = x_train/255
x_val_norm = x_val/255
x_test_norm = x_test/255

y_train = to_categorical(y_train, num_classes = 100)
y_val = to_categorical(y_val, num_classes = 100)

# VGG

A continuación se utiliza la red convolucional de 16 capas, VGG16 y se utiliza una versión pre-entrenada para la base de datos de imagenet.

In [24]:
modelVGG16 = VGG16(include_top = False, pooling = None, weights="imagenet",input_shape=(32,32,3))

In [25]:
for layer in modelVGG16.layers:
    layer.trainable=False

In [26]:
x_train_VGG16 = modelVGG16.predict(x_train_norm)
x_valid_VGG16 = modelVGG16.predict(x_val_norm)

Se prosigue a armar el modelo con las capas que se pueden visualizar a continuación

In [27]:
input_preproc = Input(shape=(1,1,512))

net = Flatten()(input_preproc)

net = Dense(units=256, activation='relu')(net)
net = Dropout(0.2)(net)
#net=Dense(units=256, activation='relu')(net)
#net=Dropout(0.2)(net)
net = Dense(units=256, activation='relu')(net)
net = Dropout(0.2)(net)
net = Dense(units=100, activation = 'softmax')(net)

model = Model(input_preproc,net)

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.summary()

In [28]:
#callbacks
checkpoint_file = 'checkpoint.VGG16.hdf5'
earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=2)
checkpointer = ModelCheckpoint(monitor="val_accuracy",filepath=checkpoint_file, verbose=1, save_best_only=True)

A continuación se entrena el modelo

In [29]:
batch_size = 64
epocs = 100

history = model.fit(x_train_VGG16, y_train,epochs = epocs, batch_size = batch_size, verbose = 1, 
        validation_data = (x_valid_VGG16, y_val), callbacks = [checkpointer, earlystop])

In [30]:
f, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize=(18,5))
ax1.plot(history.history['loss'], label="loss")
ax1.plot(history.history['val_loss'], label="val_loss")
ax1.legend()

ax2.plot(history.history['accuracy'], label="accuracy")
ax2.plot(history.history['val_accuracy'], label="val_accuracy")
ax2.legend()
plt.show();

# VGG con pesos entrenables

In [31]:
model.load_weights('checkpoint.VGG16.hdf5')

In [32]:
model_final_output = model(modelVGG16.output)
model_final = Model(modelVGG16.input,model_final_output)
model_final.summary()

In [33]:
for layer in modelVGG16.layers:
    layer.trainable=True

In [34]:
model_final.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

In [35]:
#callbacks
checkpoint_file = 'checkpoint.VGG16_fine_tunning.hdf5'
earlystop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, patience = 5, verbose = 2)
checkpointer = ModelCheckpoint(monitor = "val_accuracy",filepath = checkpoint_file, verbose = 1, save_best_only = True)

In [36]:
batch_size = 64
epocs = 100

history = model_final.fit(x_train_norm, y_train, epochs = epocs, batch_size = batch_size, verbose = 1, 
        validation_data = (x_val_norm, y_val),callbacks = [checkpointer, earlystop])

# ResNet
A continuación se utiliza ResNet 50, red convolucional de 50 capas y se utiliza una versión pre-entrenada para la base de datos de imagenet.

La utilización de redes convolucionales con pesos ya entrenados a partir de una base de datos similar se conoce como **transfer learning.**

En la siguiente celda se realiza la técnica de **data augmentation**. Cuando se cuenta con un número de imágenes pequeño, es posible aumentar el número modificando las imágenes originales (haciendo zoom,escalado, flip horizontal, etc).

In [10]:
train_datagen = ImageDataGenerator(rotation_range = 10,  zoom_range = 0.1, width_shift_range = 0.1,  
        height_shift_range = 0.1, shear_range = 0.1, horizontal_flip = True,  vertical_flip = False)
train_datagen.fit(x_train_norm)

In [11]:
resnet_model = ResNet50(
    include_top = False,
    weights = 'imagenet',
    input_shape = (224,224,3)
)

for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

Se arma el modelo de manera secuencial con las capas que se pueden observar a continuación:

In [12]:
modelRN=Sequential()
modelRN.add(UpSampling2D(size=(7, 7),interpolation='bilinear'))
modelRN.add(resnet_model)
modelRN.add(GlobalAveragePooling2D())
modelRN.add(Dropout(.25))
modelRN.add(Dense(256, activation='relu'))
modelRN.add(BatchNormalization())
modelRN.add(Dense(100, activation='softmax'))

En la siguiente celda se utiliza el optimizador SGD y se compila el modelo (tarda mucho en correr)

In [13]:
optimizer = SGD(learning_rate=1e-3, momentum=0.9)

modelRN.compile(
    optimizer = optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history=modelRN.fit(
    train_datagen.flow(x_train_norm, y_train, batch_size = 128),
    validation_data = (x_val_norm, y_val),
    epochs = 100,
    verbose = 1
)

A continuación se grafican los valores de **loss** y **val_loss** por un lado y **accuracy** y **val_accuracy** por otro, para comparar las progresiones de los distintos parámetros a lo largo del entrenamiento.

In [14]:
f, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize=(18,5))
ax1.plot(history.history['loss'], label="loss")
ax1.plot(history.history['val_loss'], label="val_loss")
ax1.legend()

ax2.plot(history.history['accuracy'], label="accuracy")
ax2.plot(history.history['val_accuracy'], label="val_accuracy")
ax2.legend()
plt.show();

In [16]:
y_test_pred = modelRN.predict(x_test_norm)
y_test_pred = y_test_pred.argmax(axis = 1)
df = pd.DataFrame(data = y_test_pred, columns = ['label'])
df.index.name = "Id"

In [17]:
df.head()

In [19]:
df.to_csv("submission.csv")

In [20]:
from IPython.display import FileLink
FileLink('submission.csv')

# Conclusiones

En primer lugar, se utilizó una red convolucional de tipo VGG con transfer learning utilzando la red VGG16 con pesos preentrenados a partir  del dataset de imagenet y se congelaron dichos pesos. Con este modelo se obtuvo un accuracy de 0.3418 para el set de datos de validación.

Dado que el valor obtenido de accuracy era bajo, se probó utilizar el mismo modelo, pero esta vez entrenando
los pesos. Con este modelo se obtuvo un accuracy de 0.5735 para los datos de validación.

Para ver si se obtenía un mejor modelo, se probó con una red convolucional de tipo Resnet con transfer
learning utilizando la red ResNet50 con pesos preentrenados a partir del dataset imagenet. Además dado que la cantidad de imagenes por clase era bastante reducida, se realizó data augmentation a los datos de train para tener una mayor cantidad de imágenes de entrada y así obtener un mejor entrenamiento del modelo. Con este modelo dentro de un modelo secuencial con otras capas se consiguió el mejor valor de accuracy de validación
el cual fue 0.7484. Además, se comprobó que no hubiera mucho overfitting ya que el valor de accuracy para train era de 0.7983, por lo que se puede decir que el modelo generaliza bastante bien.